In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import json
import warnings
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
warnings.filterwarnings("ignore")
print(tf.__version__)

2.18.0


In [3]:
!python -V

Python 3.11.12


In [4]:
!cp -r /content/drive/MyDrive/saved_model/ /content/

In [5]:
img_size = (64, 64)
save_model_path = '/content/saved_model/fabric_best_model.keras'
model = tf.keras.models.load_model(save_model_path)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ random_flip         │ (None, 64, 64, 3) │          0 │ input_layer[0][0] │
│ (RandomFlip)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ random_rotation     │ (None, 64, 64, 3) │          0 │ random_flip[0][0] │
│ (RandomRotation)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ random_zoom         │ (None, 64, 64, 3) │          0 │ random_rotation[… │
│ (RandomZoom)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 64, 64, 3) │          0 │ random_zoom[0][0] │
│ (Rescaling)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 32, 32,    │        896 │ rescaling[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 32, 32,    │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32, 32,    │          0 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 16, 16,    │          0 │ dropout[0][0]     │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 16, 16,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 16, 16,    │          0 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 8, 8, 64)  │          0 │ dropout_1[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 8, 8, 128) │     73,856 │ max_pooling2d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 8, 8, 128) │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                 

 Total params: 381,384 (1.45 MB)

 Trainable params: 126,978 (496.01 KB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 253,958 (992.03 KB)

In [6]:
label_to_class_mapping_file = '/content/saved_model/label_to_class_mapping.json'
with open(label_to_class_mapping_file, 'r') as f:
    label_to_class_mapping = json.load(f)
label_to_class_mapping

{'0': 'defect', '1': 'good'}

In [7]:
def predict_class(img_path):
    img = Image.open(img_path)
    resized_img = img.resize(img_size).convert("RGB")
    img = np.asarray(resized_img)
    img = np.expand_dims(img, axis=0)
    # img = img / 255
    prob = model.predict(img)[0]
    return round(float(np.max(prob)),2), label_to_class_mapping[str(np.argmax(prob))]

In [8]:
img_path = list(list(Path("/content/saved_model/sample_images").iterdir())[0].iterdir())[0]
prob,pred = predict_class(img_path)
print(f"Prediction: {pred} \nConfidence: {prob * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
Prediction: good 
Confidence: 99.00%


In [9]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 9.4 MB/s eta 0:00:00


In [10]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import json
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import os
import time

# Set page configuration and styling
st.set_page_config(
    page_title="Fabric Defect Detection System",
    page_icon="🛡️",
    layout="wide"
)

# Custom CSS to enhance the UI
st.markdown("""
<style>
    .main-header {
        font-size: 2.5rem;
        color: #1E3A8A;
        text-align: center;
        margin-bottom: 1.5rem;
        font-weight: 700;
    }
    .sub-header {
        font-size: 1.5rem;
        color: #1E3A8A;
        margin-top: 1rem;
        font-weight: 600;
    }
    .result-container {
        background-color: #F3F4F6;
        padding: 1.5rem;
        border-radius: 10px;
        margin-top: 1rem;
        border-left: 5px solid #1E3A8A;
    }
    .result-header {
        font-size: 1.2rem;
        font-weight: 600;
        color: #1E3A8A;
    }
    .result-text {
        font-size: 1.8rem;
        font-weight: 700;
        margin-top: 0.5rem;
    }
    .highlight-text {
        background-color: #DBEAFE;
        padding: 0.2rem 0.5rem;
        border-radius: 4px;
        font-weight: 600;
    }
    .stProgress > div > div {
        background-color: #1E3A8A;
    }
    .upload-section {
        display: flex;
        flex-direction: column;
        align-items: center;
        background-color: #F9FAFB;
        padding: 2rem;
        border-radius: 10px;
        border: 1px dashed #CBD5E1;
    }
    .footer {
        text-align: center;
        margin-top: 2rem;
        color: #6B7280;
        font-size: 0.8rem;
    }
</style>
""", unsafe_allow_html=True)

# App title and introduction
st.markdown("<h1 class='main-header'>🛡️ Fabric Defect Detection System</h1>", unsafe_allow_html=True)

st.markdown("""
<div style="text-align: center; margin-bottom: 2rem;">
    <p>Upload fabric images to detect defects and analyze quality using our advanced AI model.</p>
</div>
""", unsafe_allow_html=True)

# Constants
img_size = (64, 64)

# Paths (these would need to be updated to your actual paths)
save_model_path = '/content/saved_model/fabric_best_model.keras'
label_to_class_mapping_file = '/content/saved_model/label_to_class_mapping.json'

# Cache the model loading
@st.cache_resource
def load_model():
    try:
        model = tf.keras.models.load_model(save_model_path)
        with open(label_to_class_mapping_file, 'r') as f:
            label_to_class_mapping = json.load(f)
        return model, label_to_class_mapping
    except Exception as e:
        st.error(f"Error loading model: {e}")
        # Create a dummy model and mapping for demonstration
        st.warning("Using a demonstration model since the actual model couldn't be loaded.")
        dummy_model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(64, 64, 3)),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(4, activation='softmax')
        ])
        dummy_mapping = {"0": "good", "1": "color_defect", "2": "hole", "3": "cut"}
        return dummy_model, dummy_mapping

# Load model at app startup
with st.spinner("Loading model..."):
    model, label_to_class_mapping = load_model()
    class_to_label_mapping = {v: k for k, v in label_to_class_mapping.items()}
    class_names = list(class_to_label_mapping.keys())

def predict_class(img_path):
    img = Image.open(img_path)
    resized_img = img.resize(img_size).convert("RGB")
    img = np.asarray(resized_img)
    img = np.expand_dims(img, axis=0)
    # Normalize the image if your model expects normalized input
    # img = img / 255.0

    probs = model.predict(img)[0]
    # Convert numpy float32 to Python float
    probs = [float(p) for p in probs]
    predicted_class = label_to_class_mapping[str(np.argmax(probs))]
    return probs, predicted_class

def create_probability_plot(probs, class_names):
    # Create a horizontal bar chart using Plotly
    colors = ['#3498db' if p != max(probs) else '#e74c3c' for p in probs]

    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=class_names,
        x=[p * 100 for p in probs],  # Convert to percentage
        orientation='h',
        marker=dict(color=colors),
        text=[f"{p * 100:.2f}%" for p in probs],
        textposition='auto'
    ))

    fig.update_layout(
        title="Probability Distribution",
        xaxis_title="Probability (%)",
        yaxis_title="Defect Class",
        height=400,
        margin=dict(l=0, r=0, t=50, b=0),
        xaxis=dict(range=[0, 100]),
        yaxis=dict(categoryorder='total ascending')
    )

    return fig

def get_result_styling(predicted_class):
    """Return appropriate styling based on the predicted class."""
    if predicted_class == "good":
        return "🟢 Good", "#10B981"  # Green for good
    else:
        return "🔴 Defect Detected", "#EF4444"  # Red for defects

# Main app layout with columns
col1, col2 = st.columns([1, 1])

with col1:
    st.markdown("<h2 class='sub-header'>Upload Fabric Image</h2>", unsafe_allow_html=True)

    # File uploader
    st.markdown("<div class='upload-section'>", unsafe_allow_html=True)
    uploaded_file = st.file_uploader("", type=["jpg", "jpeg", "png"])
    st.markdown("</div>", unsafe_allow_html=True)

    # Display upload instructions
    if not uploaded_file:
        st.info("Please upload a fabric image to analyze for defects.")

        # Sample images section
        st.markdown("<h3 class='sub-header'>About Fabric Defects</h3>", unsafe_allow_html=True)
        st.markdown("""
        This system can detect the following fabric defects:
        - **Color Defects**: Inconsistencies in color or dyeing
        - **Holes**: Small to medium-sized holes in the fabric
        - **Cuts**: Linear tears or cuts in the material
        - **Good**: Fabric with no defects
        """)

with col2:
    st.markdown("<h2 class='sub-header'>Analysis Results</h2>", unsafe_allow_html=True)

    if uploaded_file:
        # Save the uploaded file temporarily
        temp_file_path = os.path.join("temp_img.jpg")
        with open(temp_file_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        # Display the uploaded image
        st.image(temp_file_path, caption="Uploaded Fabric Image")

        # Analyze with a loading animation
        with st.spinner("Analyzing fabric for defects..."):
            # Add a small delay for effect
            time.sleep(1)
            probs, predicted_class = predict_class(temp_file_path)

        # Display results
        status_text, status_color = get_result_styling(predicted_class)

        st.markdown(f"""
        <div class='result-container'>
            <div class='result-header'>Analysis Result:</div>
            <div class='result-text' style='color: {status_color};'>{status_text}</div>
            <div style='margin-top: 0.5rem;'>
                <span class='highlight-text'>Detected Class:</span> {predicted_class.replace('_', ' ').title()}
            </div>
        </div>
        """, unsafe_allow_html=True)

        # Display probability chart
        st.markdown("<div style='margin-top: 1.5rem;'>", unsafe_allow_html=True)
        fig = create_probability_plot(probs, [name.replace('_', ' ').title() for name in class_names])
        st.plotly_chart(fig, use_container_width=True)
        st.markdown("</div>", unsafe_allow_html=True)

        # Cleanup temp file
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

# Footer
st.markdown("<div class='footer'>© 2025 Fabric Defect Detection System | AI-Powered Quality Control</div>", unsafe_allow_html=True)

Writing app.py


In [14]:
from pyngrok import ngrok

ngrok_key = "2wwrghyfuHRS5ipbFGHcXbceafq_69z2V5DNkAUEPnptkBE4y"
port = 8501

ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

'https://ab82-34-106-254-116.ngrok-free.app'

In [ ]:
!rm -rf logs.txt && streamlit run /content/app.py &>/content/logs.txt